In [4]:
%pip install PyniteFEA
%pip install Pynite
%pip install trimesh
%pip install numpy matplotlib shapely trimesh scipy
%pip install pyvista
%pip install pyvistaqt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
conda activate your_environment_name
conda install -c conda-forge pynite  # Assuming it's available in conda-forge

SyntaxError: invalid syntax (2526401053.py, line 1)

In [2]:
%pip install PyniteFEA
from PyniteFEA import FEModel3D

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'PyniteFEA'

In [5]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
from shapely.geometry import Point, Polygon
import trimesh
from PyniteFEA import FEModel3D



def load_obj_model(obj_path):
    """Load an OBJ file and extract geometry"""
    print(f"Loading OBJ model from {obj_path}...")
    mesh = trimesh.load(obj_path)
    
    # Extract vertices and faces
    vertices = np.array(mesh.vertices)
    
    # Get the bounding box
    bounds = mesh.bounds
    min_corner = bounds[0]
    max_corner = bounds[1]
    
    dimensions = max_corner - min_corner
    print(f"Model dimensions: {dimensions}")
    print(f"Model bounds: {bounds}")
    
    return mesh, min_corner, max_corner

def create_building_outline(min_corner, max_corner):
    """Create a simple rectangular building outline from bounds"""
    min_x, min_y = min_corner[0], min_corner[1]
    max_x, max_y = max_corner[0], max_corner[1]
    
    # Create a simple polygon representing the building outline
    outline_points = [
        (min_x, min_y),
        (max_x, min_y),
        (max_x, max_y),
        (min_x, max_y)
    ]
    building_outline = Polygon(outline_points)
    return building_outline

def generate_column_grid(building_outline, spacing_x, spacing_y):
    """Generate a grid of columns with specified spacing"""
    bounds = building_outline.bounds
    min_x, min_y, max_x, max_y = bounds
    
    columns = []
    
    # Generate a grid of points
    for x in np.arange(min_x, max_x + 0.1, spacing_x):
        for y in np.arange(min_y, max_y + 0.1, spacing_y):
            point = Point(x, y)
            
            # Check if point is inside the building outline
            if building_outline.contains(point):
                columns.append((x, y))
    
    return columns

def create_pynite_model(columns, building_outline, floor_height=12, num_floors=3):
    """Create a Pynite structural model with the specified columns"""
    # Create a new model
    model = FEModel3D()
    
    # Define materials and sections
    model.add_material('Concrete', 3600, 1500, 0.2, 150)  # Concrete (ksi, ksi, unitless, pcf)
    model.add_section('Column24x24', 576, 27648, 27648, 46080)  # 24"x24" column
    model.add_section('Beam18x24', 432, 15552, 11664, 39916)  # 18"x24" beam
    
    # Add nodes for each column at each floor level
    column_members = []
    
    for i, (x, y) in enumerate(columns):
        # Add nodes at each floor level for this column
        for j in range(num_floors + 1):  # +1 for roof level
            node_name = f'N{i}_{j}'
            model.add_node(node_name, x, y, j * floor_height)
            
            # If not at ground level, create a column from previous level
            if j > 0:
                prev_node = f'N{i}_{j-1}'
                member_name = f'COL{i}_{j}'
                model.add_member(member_name, prev_node, node_name, 'Concrete', 'Column24x24')
                column_members.append(member_name)
    
    # Add beams between adjacent columns at each floor
    beam_members = []
    for floor in range(1, num_floors + 1):
        for i, (x1, y1) in enumerate(columns):
            for j, (x2, y2) in enumerate(columns):
                if i < j:  # Avoid duplicate beams
                    dist = np.sqrt((x2-x1)**2 + (y2-y1)**2)
                    
                    # Only connect columns that are reasonably close
                    if dist < max(1.5 * np.mean([x for x, y in columns]),
                                 1.5 * np.mean([y for x, y in columns])):
                        beam_name = f'BEAM{i}_{j}_{floor}'
                        i_node = f'N{i}_{floor}'
                        j_node = f'N{j}_{floor}'
                        model.add_member(beam_name, i_node, j_node, 'Concrete', 'Beam18x24')
                        beam_members.append(beam_name)
    
    # Create floor slabs at each level
    bounds = building_outline.bounds
    min_x, min_y, max_x, max_y = bounds
    
    for floor in range(1, num_floors + 1):
        try:
            model.add_rectangle_mesh(
                f'Floor_{floor}',
                min(5.0, (max_x - min_x) / 10),  # Mesh size
                max_x - min_x,  # Width
                max_y - min_y,  # Height
                0.5,  # Thickness (ft)
                'Concrete',  # Material
                1.0, 1.0,  # Stiffness modifiers
                (min_x, min_y, floor * floor_height),  # Origin
                'XY'  # Plane
            )
        except Exception as e:
            print(f"Error creating floor mesh at level {floor}: {e}")
    
    # Add supports at the base level
    for i in range(len(columns)):
        model.def_support(f'N{i}_0', True, True, True, True, True, True)
    
    # Add loads
    # 1. Self-weight
    model.add_member_self_weight('FZ', 1.0, 'Dead Load')
    
    # 2. Floor live load
    for quad_name in model.quads:
        model.add_quad_surface_pressure(quad_name, 0.1, 'Live Load')  # 100 psf = 0.1 ksf
    
    # Add load combination
    model.add_load_combo('Combo1', {'Dead Load': 1.2, 'Live Load': 1.6})
    
    print(f"Created Pynite model with {len(columns)} columns")
    
    return model, column_members, beam_members

def analyze_column_grid(model, column_members, beam_members):
    """Analyze the structural model and return performance metrics"""
    try:
        # Run the structural analysis
        model.analyze_linear(sparse=True)
        
        # Collect results
        results = {
            'max_deflection': 0,
            'max_column_force': 0,
            'max_beam_moment': 0,
        }
        
        # Check maximum deflection
        for node_name in model.nodes:
            if 'Combo1' in model.nodes[node_name].DZ:
                deflection = abs(model.nodes[node_name].DZ['Combo1'])
                results['max_deflection'] = max(results['max_deflection'], deflection)
        
        # Check column forces
        for member_name in column_members:
            if 'Combo1' in model.members[member_name].axial(0.5):
                axial_force = abs(model.members[member_name].axial(0.5, 'Combo1'))
                results['max_column_force'] = max(results['max_column_force'], axial_force)
        
        # Check beam moments
        for member_name in beam_members:
            if 'Combo1' in model.members[member_name].moment('Mz', 0.5):
                moment = abs(model.members[member_name].moment('Mz', 0.5, 'Combo1'))
                results['max_beam_moment'] = max(results['max_beam_moment'], moment)
        
        print(f"Analysis complete. Max deflection: {results['max_deflection']:.4f}', "
              f"Max column force: {results['max_column_force']:.2f} kips, "
              f"Max beam moment: {results['max_beam_moment']:.2f} kip-ft")
        
        return results
    
    except Exception as e:
        print(f"Analysis failed: {e}")
        return None

def visualize_2d_grid(columns, building_outline, spacing_x, spacing_y, results=None):
    """Create a 2D visualization of the column grid"""
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Get the building bounds
    bounds = building_outline.bounds
    min_x, min_y, max_x, max_y = bounds
    
    # Plot the building outline
    x, y = building_outline.exterior.xy
    ax.plot(x, y, 'k-', linewidth=2)
    
    # Plot the columns
    for i, (x, y) in enumerate(columns):
        circle = plt.Circle((x, y), 0.5, color='gray')
        ax.add_patch(circle)
        ax.text(x, y, str(i+1), ha='center', va='center', color='white')
    
    # Plot the grid lines
    for x in np.arange(min_x, max_x + 0.1, spacing_x):
        ax.axvline(x, color='lightgray', linestyle='--', alpha=0.3)
    for y in np.arange(min_y, max_y + 0.1, spacing_y):
        ax.axhline(y, color='lightgray', linestyle='--', alpha=0.3)
    
    # Set plot properties
    ax.set_xlim(min_x - 2, max_x + 2)
    ax.set_ylim(min_y - 2, max_y + 2)
    ax.set_aspect('equal')
    ax.set_xlabel('X (ft)')
    ax.set_ylabel('Y (ft)')
    
    # Add title with results if available
    if results:
        title = (f'Column Grid: {spacing_x}\' × {spacing_y}\' spacing ({len(columns)} columns)\n'
                f'Max Deflection: {results["max_deflection"]:.4f}\', '
                f'Max Column Force: {results["max_column_force"]:.2f} kips')
    else:
        title = f'Column Grid: {spacing_x}\' × {spacing_y}\' spacing ({len(columns)} columns)'
    
    ax.set_title(title)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig(f'column_grid_{spacing_x}x{spacing_y}.png')
    plt.close()
    
    print(f"Saved 2D visualization to column_grid_{spacing_x}x{spacing_y}.png")

def visualize_3d_building(columns, building_outline, results=None, num_floors=3, floor_height=12):
    """Create a 3D visualization of the building with columns"""
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Get the building bounds
    bounds = building_outline.bounds
    min_x, min_y, max_x, max_y = bounds
    
    # Plot the building outline at each floor level
    outline_coords = list(building_outline.exterior.coords)
    
    for floor in range(num_floors + 1):  # +1 for the roof
        z = floor * floor_height
        
        # Plot the outline
        x_vals = [p[0] for p in outline_coords]
        y_vals = [p[1] for p in outline_coords]
        z_vals = [z] * len(outline_coords)
        
        ax.plot(x_vals, y_vals, z_vals, 'k-', linewidth=1.5)
    
    # Add vertical lines at the corners
    for x, y in outline_coords[:-1]:  # Skip the last point (duplicate of first)
        ax.plot([x, x], [y, y], [0, num_floors * floor_height], 'k-', linewidth=1)
    
    # Plot the columns as vertical lines
    for x, y in columns:
        ax.plot([x, x], [y, y], [0, num_floors * floor_height], 'r-', linewidth=3)
        
        # Add circles at each floor level
        for floor in range(num_floors + 1):
            z = floor * floor_height
            
            # Create a circle in 3D
            theta = np.linspace(0, 2*np.pi, 20)
            radius = 0.6
            x_circle = x + radius * np.cos(theta)
            y_circle = y + radius * np.sin(theta)
            z_circle = [z] * len(theta)
            
            ax.plot(x_circle, y_circle, z_circle, 'gray')
    
    # Plot beams between adjacent columns
    for floor in range(1, num_floors + 1):
        z = floor * floor_height
        
        for i, (x1, y1) in enumerate(columns):
            for j, (x2, y2) in enumerate(columns):
                if i < j:  # Avoid duplicate beams
                    dist = np.sqrt((x2-x1)**2 + (y2-y1)**2)
                    
                    # Only connect columns that are reasonably close
                    if dist < max(1.5 * np.mean([x for x, y in columns]),
                                 1.5 * np.mean([y for x, y in columns])):
                        ax.plot([x1, x2], [y1, y2], [z, z], 'b-', linewidth=2)
    
    # Set plot properties
    ax.set_xlim(min_x - 2, max_x + 2)
    ax.set_ylim(min_y - 2, max_y + 2)
    ax.set_zlim(0, (num_floors + 0.2) * floor_height)
    ax.set_xlabel('X (ft)')
    ax.set_ylabel('Y (ft)')
    ax.set_zlabel('Z (ft)')
    
    # Add title with results if available
    if results:
        title = (f'3D Building Model ({len(columns)} columns)\n'
                f'Max Deflection: {results["max_deflection"]:.4f}\', '
                f'Max Column Force: {results["max_column_force"]:.2f} kips')
    else:
        title = f'3D Building Model ({len(columns)} columns)'
    
    ax.set_title(title)
    
    # Set a good viewing angle
    ax.view_init(elev=30, azim=-60)
    
    # Save the figure
    plt.tight_layout()
    plt.savefig('building_3d_view.png', dpi=300)
    plt.close()
    
    print("Saved 3D visualization to building_3d_view.png")

def compare_grid_options(building_outline, floor_height=12, num_floors=3):
    """Compare different column grid options"""
    options = []
    
    # Try different spacing options
    for x_spacing in [4, 6, 8]:
        for y_spacing in [4, 6, 8]:
            print(f"\nTesting {x_spacing}' × {y_spacing}' spacing:")
            
            # Generate column grid
            columns = generate_column_grid(building_outline, x_spacing, y_spacing)
            
            if len(columns) < 2:
                print(f"  Not enough columns ({len(columns)}) for this spacing, skipping")
                continue
            
            print(f"  Generated grid with {len(columns)} columns")
            
            # Create and analyze the model
            model, column_members, beam_members = create_pynite_model(
                columns, building_outline, floor_height, num_floors
            )
            
            results = analyze_column_grid(model, column_members, beam_members)
            
            if results:
                # Calculate a score (lower is better)
                area = building_outline.area
                column_density = len(columns) / area
                max_span = max(x_spacing, y_spacing)
                
                # Score based on column count, deflection, and forces
                score = (
                    10 * column_density +  # Fewer columns is better
                    30 * results['max_deflection'] / (max_span/240) +  # Normalize to span/240
                    20 * results['max_column_force'] / 100  # Normalize to 100 kips
                )
                
                options.append({
                    'x_spacing': x_spacing,
                    'y_spacing': y_spacing,
                    'columns': columns,
                    'results': results,
                    'score': score
                })
                
                # Create 2D visualization
                visualize_2d_grid(columns, building_outline, x_spacing, y_spacing, results)
            
    # Sort options by score
    options.sort(key=lambda x: x['score'])
    
    print("\nGrid Options Ranked:")
    print("Rank | Spacing    | Columns | Max Deflection | Max Column Force | Score")
    print("-" * 80)
    
    for i, option in enumerate(options):
        print(f"{i+1:4} | {option['x_spacing']}' × {option['y_spacing']}' | "
              f"{len(option['columns']):7} | "
              f"{option['results']['max_deflection']:.4f}' | "
              f"{option['results']['max_column_force']:.2f} kips | "
              f"{option['score']:.2f}")
    
    # Visualize the best option in 3D
    if options:
        best = options[0]
        print(f"\nBest option: {best['x_spacing']}' × {best['y_spacing']}' spacing with {len(best['columns'])} columns")
        visualize_3d_building(best['columns'], building_outline, best['results'], num_floors, floor_height)
    else:
        print("\nNo valid options found.")
    
    return options

def run_optimization(obj_path, floor_height=12, num_floors=3):
    """Run the full optimization process"""
    # Load the model
    mesh, min_corner, max_corner = load_obj_model(obj_path)
    
    # Create a building outline from the model bounds
    building_outline = create_building_outline(min_corner, max_corner)
    print(f"Created building outline with area: {building_outline.area:.2f} sq ft")
    
    # Compare different grid options
    options = compare_grid_options(building_outline, floor_height, num_floors)
    
    return options

# Example usage
if __name__ == "__main__":
    obj_path = r"..\Reference Files\rectangles3d.obj"  # Update with your actual OBJ file path
    
    options = run_optimization(
        obj_path,
        floor_height=12,  # Floor height in feet
        num_floors=3      # Number of floors
    )

ModuleNotFoundError: No module named 'PyniteFEA'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import trimesh
from shapely.geometry import Point, Polygon
import os
from Pynite import FEModel3D

class StructuralOptimizer:
    """Class for optimizing column and beam placement from an OBJ model using Pynite"""
    
    def __init__(self, obj_file):
        """Initialize with the OBJ file path"""
        self.obj_file = obj_file
        self.mesh = None
        self.building_outline = None
        self.min_corner = None
        self.max_corner = None
        self.best_grid = None
        self.best_model = None
        self.analysis_results = {}
        
    def load_obj(self):
        """Load the OBJ file and extract basic information"""
        print(f"Loading OBJ file: {self.obj_file}")
        try:
            self.mesh = trimesh.load(self.obj_file)
            print("OBJ file successfully loaded.")
        except Exception as e:
            print(f"Error loading OBJ file: {e}")
            self.mesh = None
file)
            
            # Get model bounds
            self.bounds = self.mesh.bounds
            self.min_corner = self.bounds[0]
            self.max_corner = self.bounds[1]
            
            dimensions = self.max_corner - self.min_corner
            print(f"Model dimensions: {dimensions}")
            print(f"Model bounds: {self.bounds}")
            
    def extract_building_outline(self):
        """Extract building outline from the OBJ model"""
        if self.mesh is None:
            print("Mesh not loaded, cannot extract building outline.")
            return
                
        # Project all vertices to XY plane
        vertices_2d = np.array(self.mesh.vertices)[:, :2]
        
        # Create a simple convex hull as the building outline
        from scipy.spatial import ConvexHull
        try:
            hull = ConvexHull(vertices_2d)
            hull_points = vertices_2d[hull.vertices]
            self.building_outline = Polygon(hull_points)
            print(f"Extracted building outline with area: {self.building_outline.area:.2f} sq.ft.")
            
            return True
        except Exception as e:
            print(f"Error extracting building outline: {e}")
            
            # Fall back to rectangular outline
            min_x, min_y = vertices_2d.min(axis=0)
            max_x, max_y = vertices_2d.max(axis=0)
            
            rect_points = [(min_x, min_y), (max_x, min_y), 
                          (max_x, max_y), (min_x, max_y)]
            self.building_outline = Polygon(rect_points)
            print(f"Using rectangular building outline with area: {self.building_outline.area:.2f} sq.ft.")
            
            return True
   print(f"Error extracting building outline: {e}")
            
            # Fall back to rectangular outline
            min_x, min_y = self.min_corner[0], self.min_corner[1]
            max_x, max_y = self.max_corner[0], self.max_corner[1]
            
            rect_points = [(min_x, min_y), (max_x, min_y), 
                          (max_x, max_y), (min_x, max_y)]
            self.building_outline = Polygon(rect_points)
            print(f"Using rectangular building outline with area: {self.building_outline.area:.2f} sq.ft.")
            
            return True
            
    def generate_column_grid(self, spacing_x, spacing_y, offset_x=0, offset_y=0):
        """Generate a grid of column locations with the specified spacing"""
        if self.building_outline is None:
            if not self.extract_building_outline():
                return []
                
        # Get bounds of the building
        bounds = self.building_outline.bounds
        min_x, min_y, max_x, max_y = bounds
        
        # Generate grid points with offset
        columns = []
        for x in np.arange(min_x + offset_x, max_x + 0.1, spacing_x):
            for y in np.arange(min_y + offset_y, max_y + 0.1, spacing_y):
                point = Point(x, y)
                
                # Check if point is inside the building outline
                if self.building_outline.contains(point):
                    columns.append((x, y))
        
        return columns
        
    def create_pynite_model(self, columns, floor_height=12, num_floors=3):
        """Create a Pynite structural model with the specified columns"""
        # Create a new model
        model = FEModel3D()
        
        # Define materials
        model.add_material('Concrete', 3600, 1500, 0.2, 150)  # Concrete material (ksi)
        
        # Add sections
        model.add_section('Column24x24', 576, 27648, 27648, 46080)  # 24" square column
        model.add_section('Beam18x24', 432, 15552, 11664, 39916)  # 18"x24" beam
        
        # Add nodes for each column at each floor level
        column_members = []
        beam_members = []
        floor_nodes = []
        
        # Store nodes by floor
        for floor in range(num_floors + 1):
            floor_nodes.append([])
        
        # Create nodes and columns
        for i, (x, y) in enumerate(columns):
            for floor in range(num_floors + 1):
                z = floor * floor_height
                node_name = f'N{i}_{floor}'
                model.add_node(node_name, x, y, z)
                floor_nodes[floor].append((i, node_name))
                
                # Add column from previous floor (if not at ground level)
                if floor > 0:
                    prev_node = f'N{i}_{floor-1}'
                    column_name = f'C{i}_{floor}'
                    model.add_member(column_name, prev_node, node_name, 'Concrete', 'Column24x24')
                    column_members.append(column_name)
        
        # Add beams between columns at each floor level (using closest neighbors)
        for floor in range(1, num_floors + 1):
            # Create distance matrix between all columns
            coords = np.array(columns)
            num_columns = len(columns)
            
            for i in range(num_columns):
                # Find the nearest columns to connect with beams
                distances = []
                for j in range(num_columns):
                    if i != j:
                        dist = np.sqrt((columns[i][0] - columns[j][0])**2 + 
                                      (columns[i][1] - columns[j][1])**2)
                        distances.append((j, dist))
                
                # Sort by distance and connect to the closest columns
                distances.sort(key=lambda x: x[1])
                max_connections = min(4, len(distances))  # Connect to at most 4 neighbors
                
                for j in range(max_connections):
                    neigh_idx, dist = distances[j]
                    
                    # Only add beam if it doesn't already exist
                    if i < neigh_idx:  # Avoid duplicates
                        i_node = f'N{i}_{floor}'
                        j_node = f'N{neigh_idx}_{floor}'
                        beam_name = f'B{i}_{neigh_idx}_{floor}'
                        
                        model.add_member(beam_name, i_node, j_node, 'Concrete', 'Beam18x24')
                        beam_members.append(beam_name)
        
        # Add floor slabs (simplified as a mesh)
        try:
            bounds = self.building_outline.bounds
            min_x, min_y, max_x, max_y = bounds
            
            for floor in range(1, num_floors + 1):
                mesh_name = model.add_rectangle_mesh(
                    f'Floor_{floor}',
                    min(5.0, max((max_x - min_x) / 10, (max_y - min_y) / 10)),  # Mesh size
                    max_x - min_x,  # Width
                    max_y - min_y,  # Height
                    0.5,  # Thickness (ft)
                    'Concrete',  # Material
                    1.0, 1.0,  # Stiffness modifiers
                    (min_x, min_y, floor * floor_height),  # Origin
                    'XY'  # Plane
                )
        except Exception as e:
            print(f"Warning: Could not create floor mesh: {e}")
            print("Continuing without floor mesh...")
        
        # Add supports at the base
        for i, _ in enumerate(columns):
            model.def_support(f'N{i}_0', True, True, True, True, True, True)
        
        # Add loads
        # 1. Self-weight
        model.add_member_self_weight('FZ', 1.0, 'Dead Load')
        
        # 2. Floor live load
        for quad_name in model.quads:
            model.add_quad_surface_pressure(quad_name, 0.1, 'Live Load')  # 100 psf = 0.1 ksf
        
        # Add load combination
        model.add_load_combo('Combo1', {'Dead Load': 1.2, 'Live Load': 1.6})
        
        print(f"Created Pynite model with {len(columns)} columns and {len(beam_members)} beams")
        
        return model, column_members, beam_members
    
    def analyze_model(self, model, column_members, beam_members):
        """Analyze the structural model and return performance metrics"""
        try:
            # Run the structural analysis
            print("Running structural analysis...")
            model.analyze_linear(sparse=True, check_stability=True)
            
            # Collect results
            results = {
                'max_deflection': 0,
                'max_column_force': 0,
                'max_beam_moment': 0,
                'max_beam_shear': 0
            }
            
            # Find maximum deflection
            for node_name in model.nodes:
                if 'Combo1' in model.nodes[node_name].DZ:
                    deflection = abs(model.nodes[node_name].DZ['Combo1'])
                    results['max_deflection'] = max(results['max_deflection'], deflection)
            
            # Find maximum column force
            for member_name in column_members:
                if 'Combo1' in model.members[member_name].axial(0.5):
                    axial_force = abs(model.members[member_name].axial(0.5, 'Combo1'))
                    results['max_column_force'] = max(results['max_column_force'], axial_force)
            
            # Find maximum beam moment and shear
            for member_name in beam_members:
                if 'Combo1' in model.members[member_name].moment('Mz', 0.5):
                    moment = abs(model.members[member_name].moment('Mz', 0.5, 'Combo1'))
                    results['max_beam_moment'] = max(results['max_beam_moment'], moment)
                
                if 'Combo1' in model.members[member_name].shear('Fy', 0.5):
                    shear = abs(model.members[member_name].shear('Fy', 0.5, 'Combo1'))
                    results['max_beam_shear'] = max(results['max_beam_shear'], shear)
            
            print(f"Analysis complete: Max deflection = {results['max_deflection']:.4f}', "
                  f"Max column force = {results['max_column_force']:.2f} kips, "
                  f"Max beam moment = {results['max_beam_moment']:.2f} kip-ft")
            
            return results
            
        except Exception as e:
            print(f"Analysis failed: {e}")
            return None
    
    def optimize_grid(self, min_spacing=4, max_spacing=10, spacing_step=2, 
                     floor_height=12, num_floors=3):
        """Find the optimal column grid spacing"""
        if self.building_outline is None:
            if not self.extract_building_outline():
                return None
        
        best_score = float('inf')
        best_option = None
        best_model = None
        best_results = None
        
        options = []
        
        # Try different spacings
        for x_spacing in np.arange(min_spacing, max_spacing + 0.1, spacing_step):
            for y_spacing in np.arange(min_spacing, max_spacing + 0.1, spacing_step):
                # Try different offsets for better column placement
                offset_variations = [
                    (0, 0),  # No offset
                    (x_spacing/2, 0),  # Half offset in x
                    (0, y_spacing/2),  # Half offset in y
                    (x_spacing/2, y_spacing/2)  # Half offset in both
                ]
                
                for offset_x, offset_y in offset_variations:
                    print(f"\nTesting {x_spacing}' × {y_spacing}' grid with offset ({offset_x:.1f}, {offset_y:.1f}):")
                    
                    # Generate column grid
                    columns = self.generate_column_grid(x_spacing, y_spacing, offset_x, offset_y)
                    
                    if len(columns) < 4:
                        print(f"  Not enough columns ({len(columns)}), skipping...")
                        continue
                    
                    print(f"  Generated grid with {len(columns)} columns")
                    
                    # Create Pynite model
                    model, column_members, beam_members = self.create_pynite_model(
                        columns, floor_height, num_floors
                    )
                    
                    # Analyze the model
                    results = self.analyze_model(model, column_members, beam_members)
                    
                    if results:
                        # Calculate score based on structural performance and economy
                        # Lower score is better
                        column_count_factor = len(columns) / 10  # Normalize by assuming 10 columns is middle ground
                        
                        area = self.building_outline.area
                        max_span = max(x_spacing, y_spacing)
                        
                        # Calculate span-to-depth ratio for beams (24" deep beams)
                        span_depth_ratio = (max_span * 12) / 24
                        
                        # Check span-to-depth ratio (should be below 20 for typical concrete beams)
                        span_penalty = max(0, span_depth_ratio / 20 - 1) * 10
                        
                        # Check deflection ratio (L/360 is common for office/residential)
                        deflection_limit = max_span * 12 / 360  # in inches
                        deflection_ratio = results['max_deflection'] * 12 / deflection_limit
                        deflection_penalty = max(0, deflection_ratio - 1) * 20
                        
                        # Final score calculation
                        score = (
                            column_count_factor * 30 +  # Economy (30% weight)
                            span_penalty * 30 +         # Constructability (30% weight)
                            deflection_ratio * 40       # Performance (40% weight)
                        )
                        
                        options.append({
                            'x_spacing': x_spacing,
                            'y_spacing': y_spacing,
                            'offset_x': offset_x,
                            'offset_y': offset_y,
                            'columns': columns,
                            'results': results,
                            'score': score,
                            'deflection_ratio': deflection_ratio,
                            'span_depth_ratio': span_depth_ratio
                        })
                        
                        print(f"  Score: {score:.2f} (Deflection ratio: {deflection_ratio:.2f}, "
                              f"Span/depth: {span_depth_ratio:.2f})")
                        
                        # Update best option if this is better
                        if score < best_score:
                            best_score = score
                            best_option = {
                                'x_spacing': x_spacing,
                                'y_spacing': y_spacing,
                                'offset_x': offset_x,
                                'offset_y': offset_y,
                                'columns': columns,
                                'column_count': len(columns),
                                'score': score
                            }
                            best_model = model
                            best_results = results
        
        # Sort options by score
        options.sort(key=lambda x: x['score'])
        
        # Print top options
        print("\nTop Column Grid Options:")
        print("Rank | Spacing | Offset | Columns | Defl Ratio | Span/Depth | Score")
        print("-" * 75)
        
        for i, option in enumerate(options[:5]):
            print(f"{i+1:4} | {option['x_spacing']:.1f}' × {option['y_spacing']:.1f}' | "
                  f"({option['offset_x']:.1f}, {option['offset_y']:.1f}) | "
                  f"{len(option['columns']):7} | "
                  f"{option['deflection_ratio']:10.2f} | "
                  f"{option['span_depth_ratio']:10.2f} | "
                  f"{option['score']:6.2f}")
        
        # Save the best option
        self.best_grid = best_option
        self.best_model = best_model
        self.analysis_results = best_results
        
        # Create visualizations for the best option
        if best_option:
            self.visualize_2d_grid(best_option['columns'], best_option['x_spacing'], 
                                  best_option['y_spacing'], best_results)
            self.visualize_3d_building(best_option['columns'], num_floors, floor_height, best_results)
        
        return options
    
    def visualize_2d_grid(self, columns, spacing_x, spacing_y, results=None):
        """Create a 2D visualization of the column grid"""
        if self.building_outline is None:
            if not self.extract_building_outline():
                return
        
        fig, ax = plt.subplots(figsize=(12, 9))
        
        # Get the building bounds
        bounds = self.building_outline.bounds
        min_x, min_y, max_x, max_y = bounds
        
        # Plot the building outline
        x, y = self.building_outline.exterior.xy
        ax.plot(x, y, 'k-', linewidth=2, label='Building Outline')
        
        # Add grid lines
        for x in np.arange(min_x - spacing_x, max_x + spacing_x*2, spacing_x):
            ax.axvline(x, color='lightgray', linestyle='--', alpha=0.5)
        for y in np.arange(min_y - spacing_y, max_y + spacing_y*2, spacing_y):
            ax.axhline(y, color='lightgray', linestyle='--', alpha=0.5)
        
        # Plot the columns
        column_x = [col[0] for col in columns]
        column_y = [col[1] for col in columns]
        ax.scatter(column_x, column_y, s=100, color='blue', marker='s', label='Columns')
        
        # Add column labels
        for i, (x, y) in enumerate(columns):
            ax.text(x, y, f'{i+1}', ha='center', va='center', color='white', fontweight='bold')
            
        # Plot beam connections
        if len(columns) > 1:
            coords = np.array(columns)
            for i, (x1, y1) in enumerate(columns):
                # Find nearest columns
                distances = []
                for j, (x2, y2) in enumerate(columns):
                    if i != j:
                        dist = np.sqrt((x2-x1)**2 + (y2-y1)**2)
                        distances.append((j, dist))
                
                # Sort by distance and draw beams to the closest columns
                distances.sort(key=lambda x: x[1])
                max_connections = min(4, len(distances))
                
                for j in range(max_connections):
                    neigh_idx, dist = distances[j]
                    # Only draw each beam once
                    if i < neigh_idx:
                        x2, y2 = columns[neigh_idx]
                        ax.plot([x1, x2], [y1, y2], 'r-', alpha=0.7, linewidth=2)
                        # Add span distance label
                        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
                        ax.text(mid_x, mid_y, f'{dist:.1f}\'', 
                               ha='center', va='center', fontsize=8, 
                               bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.1'))
        
        # Add legend
        ax.legend(loc='upper right')
        
        # Set plot properties
        ax.set_xlim(min_x - spacing_x, max_x + spacing_x)
        ax.set_ylim(min_y - spacing_y, max_y + spacing_y)
        ax.set_aspect('equal')
        ax.set_xlabel('X (ft)')
        ax.set_ylabel('Y (ft)')
        
        # Add title with results
        if results:
            title = (f'Optimal Column Grid: {spacing_x:.1f}\' × {spacing_y:.1f}\' spacing\n'
                    f'{len(columns)} columns, Max Deflection: {results["max_deflection"]:.4f}\', '
                    f'Max Column Force: {results["max_column_force"]:.2f} kips')
        else:
            title = f'Column Grid: {spacing_x:.1f}\' × {spacing_y:.1f}\' spacing ({len(columns)} columns)'
        
        ax.set_title(title)
        
        # Save the figure
        plt.tight_layout()
        plt.savefig('optimal_column_grid_2d.png', dpi=300)
        plt.close()
        
        print("Saved 2D visualization to optimal_column_grid_2d.png")
    
    def visualize_3d_building(self, columns, num_floors=3, floor_height=12, results=None):
        """Create a 3D visualization of the building with columns and beams"""
        if self.building_outline is None:
            if not self.extract_building_outline():
                return
        
        fig = plt.figure(figsize=(14, 10))
        ax = fig.add_subplot(111, projection='3d')
        
        # Get the building bounds
        bounds = self.building_outline.bounds
        min_x, min_y, max_x, max_y = bounds
        
        # Plot the building outline at each floor
        if hasattr(self.building_outline, 'exterior'):
            outline_points = list(self.building_outline.exterior.coords)
        else:
            # Create a simple rectangle if outline doesn't have exterior attribute
            outline_points = [(min_x, min_y), (max_x, min_y), 
                             (max_x, max_y), (min_x, max_y), (min_x, min_y)]
        
        # Draw building outline at each level
        for floor in range(num_floors + 1):
            z = floor * floor_height
            x_vals = [p[0] for p in outline_points]
            y_vals = [p[1] for p in outline_points]
            z_vals = [z] * len(outline_points)
            
            ax.plot(x_vals, y_vals, z_vals, 'k-', linewidth=1.5, alpha=0.7)
            
            # Add vertical lines at corners
            if floor < num_floors:
                for i in range(len(outline_points) - 1):  # Skip last point (duplicate of first)
                    x, y = outline_points[i]
                    ax.plot([x, x], [y, y], [z, z + floor_height], 'k-', linewidth=1, alpha=0.5)
        
        # Add floor slabs (semi-transparent)
        floor_colors = ['lightskyblue', 'skyblue', 'deepskyblue', 'royalblue']
        
        for floor in range(1, num_floors + 1):  # Skip ground floor
            z = floor * floor_height
            
            # Create vertices for the floor slab
            verts = [(x, y, z) for x, y in outline_points[:-1]]  # Skip last point (duplicate)
            
            # Create polygon for the floor
            poly = Poly3DCollection([verts], alpha=0.3)
            poly.set_facecolor(floor_colors[floor % len(floor_colors)])
            poly.set_edgecolor('black')
            ax.add_collection3d(poly)
        
        # Plot columns
        column_color = 'darkgray'
        column_width = 1.5  # Width of columns
        column_half_width = column_width / 2
        
        for i, (x, y) in enumerate(columns):
            # Draw column as vertical line
            ax.plot([x, x], [y, y], [0, num_floors * floor_height], 
                    color=column_color, linewidth=6, alpha=0.8)
            
            # Add column number label at the top
            ax.text(x, y, num_floors * floor_height + 1, f'{i+1}', 
                   ha='center', va='center', fontsize=9)
            
            # Draw column sections at each floor as rectangles
            for floor in range(num_floors + 1):  # Including roof
                z = floor * floor_height
                
                # Draw column cross-section
                for dx, dy in [(1, 1), (1, -1), (-1, -1), (-1, 1), (1, 1)]:
                    ax.plot([x + dx*column_half_width, x + dx*column_half_width], 
                           [y + dy*column_half_width, y + dy*column_half_width], 
                           [z, z], color=column_color, linewidth=2)
        
        # Plot beams between columns
        beam_color = 'brown'
        if len(columns) > 1:
            # Calculate distances between all columns
            for i, (x1, y1) in enumerate(columns):
                # Find nearest columns
                distances = []
                for j, (x2, y2) in enumerate(columns):
                    if i != j:
                        dist = np.sqrt((x2-x1)**2 + (y2-y1)**2)
                        distances.append((j, dist))
                
                # Sort by distance and draw beams to the closest columns
                distances.sort(key=lambda x: x[1])
                max_connections = min(4, len(distances))
                
                for j in range(max_connections):
                    neigh_idx, dist = distances[j]
                    # Only draw each beam once
                    if i < neigh_idx:
                        x2, y2 = columns[neigh_idx]
                        
                        # Draw beams at each floor level
                        for floor in range(1, num_floors + 1):  # Skip ground floor
                            z = floor * floor_height
                            beam_z = z - 0.5  # Draw beam slightly below floor level
                            
                            ax.plot([x1, x2], [y1, y2], [beam_z, beam_z], 
                                    color=beam_color, linewidth=4, alpha=0.7)
                            
                            # Add span label in the middle of the beam
                            mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
                            if floor == 1:  # Only add label on the first floor to avoid clutter
                                ax.text(mid_x, mid_y, beam_z - 0.5, f'{dist:.1f}\'', 
                                       ha='center', va='top', fontsize=8)
        
        # Set plot properties
        ax.set_xlim(min_x - 2, max_x + 2)
        ax.set_ylim(min_y - 2, max_y + 2)
        ax.set_zlim(0, (num_floors + 0.2) * floor_height)
        ax.set_xlabel('X (ft)')
        ax.set_ylabel('Y (ft)')
        ax.set_zlabel('Z (ft)')
        
        # Add title with results
        if results:
            title = (f'3D Structural Model ({len(columns)} columns)\n'
                    f'Max Deflection: {results["max_deflection"]:.4f}\', '
                    f'Max Column Force: {results["max_column_force"]:.2f} kips, '
                    f'Max Beam Moment: {results["max_beam_moment"]:.2f} kip-ft')
        else:
            title = f'3D Structural Model ({len(columns)} columns)'
        
        ax.set_title(title)
        
        # Set a good viewing angle
        ax.view_init(elev=30, azim=-60)
        
        # Save the figure
        plt.tight_layout()
        plt.savefig('optimal_building_3d.png', dpi=300)
        plt.close()
        
        print("Saved 3D visualization to optimal_building_3d.png")
    
    def export_to_cad(self, filename="optimal_structure.dxf"):
        """Export the optimal column and beam layout to a DXF file for CAD software"""
        if self.best_grid is None:
            print("No optimized grid to export. Run optimize_grid() first.")
            return False
        
        try:
            import ezdxf
            
            # Create a new DXF document
            doc = ezdxf.new('R2010')
            
            # Add a new modelspace
            msp = doc.modelspace()
            
            # Create layers
            doc.layers.new(name='Building_Outline', dxfattribs={'color': 7})  # White
            doc.layers.new(name='Columns', dxfattribs={'color': 1})  # Red
            doc.layers.new(name='Beams', dxfattribs={'color': 5})  # Blue
            doc.layers.new(name='Dimensions', dxfattribs={'color': 3})  # Green
            
            # Add building outline
            if self.building_outline:
                outline_points = list(self.building_outline.exterior.coords)
                msp.add_lwpolyline(outline_points, dxfattribs={'layer': 'Building_Outline'})
            
            # Add columns (as rectangles)
            column_size = 2.0  # 2' square columns
            half_size = column_size / 2
            
            for i, (x, y) in enumerate(self.best_grid['columns']):
                # Draw column as rectangle

In [ ]:
from PyNite import FEModel3D

# Create a new finite element model
model = FEModel3D()

# Define a default material (can be customized)
model.add_material('Steel', E=29000, G=11500, nu=0.3, rho=0.284)

# Create nodes from unique coordinates
model.add_node('N1', -7.17073392868042, 6.841569900512695, 0) # Also represents: N15, N17
model.add_node('N2', -7.17073392868042, 6.841569900512695, 5) # Also represents: N16, N21
model.add_node('N3', 1.2679941654205322, 6.841569900512695, 0) # Also represents: N5, N18, N25, N39, N41
model.add_node('N4', 1.2679941654205322, 6.841569900512695, 5) # Also represents: N6, N22, N26, N40, N45
model.add_node('N7', 1.2679941654205322, 11.352361679077148, 0) # Also represents: N9, N19
model.add_node('N8', 1.2679941654205322, 11.352361679077148, 5) # Also represents: N10, N23
model.add_node('N11', -7.17073392868042, 11.352361679077148, 0) # Also represents: N13, N20
model.add_node('N12', -7.17073392868042, 11.352361679077148, 5) # Also represents: N14, N24
model.add_node('N27', 3.2604687213897705, 6.841569900512695, 0) # Also represents: N29, N42, N151, N153, N163
model.add_node('N28', 3.2604687213897705, 6.841569900512695, 5) # Also represents: N30, N46, N152, N154, N167
model.add_node('N31', 3.2604687213897705, 10.458905220031738, 0) # Also represents: N33, N43
model.add_node('N32', 3.2604687213897705, 10.458905220031738, 5) # Also represents: N34, N47
model.add_node('N35', 1.2679941654205322, 10.458905220031738, 0) # Also represents: N37, N44
model.add_node('N36', 1.2679941654205322, 10.458905220031738, 5) # Also represents: N38, N48
model.add_node('N49', 3.2604687213897705, 5.641730785369873, 0) # Also represents: N63, N65, N83, N85, N92
model.add_node('N50', 3.2604687213897705, 5.641730785369873, 5) # Also represents: N64, N69, N84, N86, N96
model.add_node('N51', 6.271378040313721, 5.641730785369873, 0) # Also represents: N53, N66
model.add_node('N52', 6.271378040313721, 5.641730785369873, 5) # Also represents: N54, N70
model.add_node('N55', 6.271378040313721, 8.650237083435059, 0) # Also represents: N57, N67
model.add_node('N56', 6.271378040313721, 8.650237083435059, 5) # Also represents: N58, N71
model.add_node('N59', 3.2604687213897705, 8.650237083435059, 0) # Also represents: N61, N68
model.add_node('N60', 3.2604687213897705, 8.650237083435059, 5) # Also represents: N62, N72
model.add_node('N73', 3.2604687213897705, 3.831695556640625, 0) # Also represents: N87, N89, N107, N109, N116, N127, N129, N139, N147, N149, N162
model.add_node('N74', 3.2604687213897705, 3.831695556640625, 5) # Also represents: N88, N93, N108, N110, N120, N128, N130, N143, N148, N150, N166
model.add_node('N75', 8.351167678833008, 3.831695556640625, 0) # Also represents: N77, N90
model.add_node('N76', 8.351167678833008, 3.831695556640625, 5) # Also represents: N78, N94
model.add_node('N79', 8.351167678833008, 5.641730785369873, 0) # Also represents: N81, N91
model.add_node('N80', 8.351167678833008, 5.641730785369873, 5) # Also represents: N82, N95
model.add_node('N97', 3.2604687213897705, 1.3467525243759155, 0) # Also represents: N111, N113
model.add_node('N98', 3.2604687213897705, 1.3467525243759155, 5) # Also represents: N112, N117
model.add_node('N99', 5.686288356781006, 1.3467525243759155, 0) # Also represents: N101, N114
model.add_node('N100', 5.686288356781006, 1.3467525243759155, 5) # Also represents: N102, N118
model.add_node('N103', 5.686288356781006, 3.831695556640625, 0) # Also represents: N105, N115
model.add_node('N104', 5.686288356781006, 3.831695556640625, 5) # Also represents: N106, N119
model.add_node('N121', 0, 0, 0) # Also represents: N135, N137
model.add_node('N122', 0, 0, 5) # Also represents: N136, N141
model.add_node('N123', 3.2604687213897705, 0, 0) # Also represents: N125, N138
model.add_node('N124', 3.2604687213897705, 0, 5) # Also represents: N126, N142
model.add_node('N131', 0, 3.831695556640625, 0) # Also represents: N133, N140, N145, N159, N161
model.add_node('N132', 0, 3.831695556640625, 5) # Also represents: N134, N144, N146, N160, N165
model.add_node('N155', 0, 6.841569900512695, 0) # Also represents: N157, N164, N175, N177, N187
model.add_node('N156', 0, 6.841569900512695, 5) # Also represents: N158, N168, N176, N178, N191
model.add_node('N169', -1.7790435552597046, 4.971345901489258, 0) # Also represents: N183, N185
model.add_node('N170', -1.7790435552597046, 4.971345901489258, 5) # Also represents: N184, N189
model.add_node('N171', 0, 4.971345901489258, 0) # Also represents: N173, N186, N199, N201, N211
model.add_node('N172', 0, 4.971345901489258, 5) # Also represents: N174, N190, N200, N202, N215
model.add_node('N179', -1.7790435552597046, 6.841569900512695, 0) # Also represents: N181, N188
model.add_node('N180', -1.7790435552597046, 6.841569900512695, 5) # Also represents: N182, N192
model.add_node('N193', -2.560492515563965, 1.9951889514923096, 0) # Also represents: N207, N209
model.add_node('N194', -2.560492515563965, 1.9951889514923096, 5) # Also represents: N208, N213
model.add_node('N195', 0, 1.9951889514923096, 0) # Also represents: N197, N210
model.add_node('N196', 0, 1.9951889514923096, 5) # Also represents: N198, N214
model.add_node('N203', -2.560492515563965, 4.971345901489258, 0) # Also represents: N205, N212
model.add_node('N204', -2.560492515563965, 4.971345901489258, 5) # Also represents: N206, N216

# Print summary
print(f"Created 54 nodes with unique coordinates out of 216 total node entries")

# Create a mapping dictionary for future reference
node_mapping = {
    # Original nodes mapped to the nodes actually created in the model
    'N1': 'N1', 'N15': 'N1', 'N17': 'N1',
    'N2': 'N2', 'N16': 'N2', 'N21': 'N2',
    'N3': 'N3', 'N5': 'N3', 'N18': 'N3', 'N25': 'N3', 'N39': 'N3', 'N41': 'N3',
    'N4': 'N4', 'N6': 'N4', 'N22': 'N4', 'N26': 'N4', 'N40': 'N4', 'N45': 'N4',
    # Additional mappings would continue...
}

# Example of how to continue with the model
print("\nNode creation complete. To proceed with your model:")
print("1. Add sections (beams, columns)")
print("2. Create members between nodes")
print("3. Define supports")
print("4. Apply loads")
print("5. Define load combinations")
print("6. Run analysis")

# For visualization, you could export to a format like this:
print("\nNode coordinates for visualization:")
for node_name, node_obj in model.nodes.items():
    print(f"{node_name}: ({node_obj.X}, {node_obj.Y}, {node_obj.Z})")